In [0]:
print("Student Performance Analysis Notebook")
print("Rows analyzed:", len(df))
print("Overall average score:", df["average_score"].mean().round(2))
print("Excellent:", (df['performance'] == 'Excellent').sum(),
      "Good:", (df['performance'] == 'Good').sum(),
      "Needs Improvement:", (df['performance'] == 'Needs Improvement').sum())


Student Performance Analysis Notebook
Rows analyzed: 15
Overall average score: 80.38
Excellent: 7 Good: 5 Needs Improvement: 3


In [0]:
spark_df = spark.table("retail.default.students")
display(spark_df.limit(20))

name,gender,math_score,reading_score,writing_score,study_hours
Riya,female,88,92,85,5
Arjun,male,76,70,72,3
Priya,female,90,95,93,6
Rahul,male,65,60,58,2
Ananya,female,82,85,88,4
Vikram,male,58,64,62,2
Meera,female,94,91,89,6
Karan,male,70,75,72,3
Sneha,female,80,83,81,4
Dev,male,85,78,82,5


In [0]:
import pandas as pd
try:
    df
except NameError:
    df = spark_df.toPandas()

# Basic checks
print("Rows:", len(df))
print("Columns and dtypes:\n", df.dtypes)
display(df.isnull().sum())

# Convert score columns to numeric (defensive)
score_cols = ["math_score", "reading_score", "writing_score", "study_hours"]
for c in score_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Drop rows with missing essential data (if any)
df = df.dropna(subset=score_cols).reset_index(drop=True)
display(df.head())


Rows: 15
Columns and dtypes:
 name             object
gender           object
math_score        int64
reading_score     int64
writing_score     int64
study_hours       int64
dtype: object


name             0
gender           0
math_score       0
reading_score    0
writing_score    0
study_hours      0
dtype: int64

name,gender,math_score,reading_score,writing_score,study_hours
Riya,female,88,92,85,5
Arjun,male,76,70,72,3
Priya,female,90,95,93,6
Rahul,male,65,60,58,2
Ananya,female,82,85,88,4


In [0]:
import numpy as np

# Average of three subject scores
df["average_score"] = df[["math_score","reading_score","writing_score"]].mean(axis=1).round(2)

# Performance label
df["performance"] = np.where(df["average_score"] >= 85, "Excellent",
                     np.where(df["average_score"] >= 70, "Good", "Needs Improvement"))

display(df.head())


name,gender,math_score,reading_score,writing_score,study_hours,average_score,performance
Riya,female,88,92,85,5,88.33,Excellent
Arjun,male,76,70,72,3,72.67,Good
Priya,female,90,95,93,6,92.67,Excellent
Rahul,male,65,60,58,2,61.0,Needs Improvement
Ananya,female,82,85,88,4,85.0,Excellent


In [0]:
# Basic descriptive stats
display(df[["math_score","reading_score","writing_score","average_score","study_hours"]].describe())

# Average score by gender
avg_by_gender = df.groupby("gender")["average_score"].mean().round(2)
print("Average Score by Gender:")
display(avg_by_gender)

# Counts by performance category
performance_counts = df["performance"].value_counts()
print("Performance distribution:")
display(performance_counts)


math_score,reading_score,writing_score,average_score,study_hours
15.0,15.0,15.0,15.0,15.0
80.0,81.2,79.93333333333334,80.37733333333334,4.2
12.118462891684855,12.30098718454278,12.062140692896852,11.962534052297654,1.65615734242165
58.0,60.0,58.0,61.0,2.0
73.0,72.5,72.0,72.5,3.0
82.0,83.0,82.0,81.67,4.0
89.0,91.5,88.5,89.83,5.5
95.0,98.0,96.0,96.33,7.0


Average Score by Gender:


gender
female    88.54
male      71.05
Name: average_score, dtype: float64

Performance distribution:


performance
Excellent            7
Good                 5
Needs Improvement    3
Name: count, dtype: int64

In [0]:
# Top performer
top = df.loc[df["average_score"].idxmax()].to_dict()
print("Top student:", top["name"], "Avg:", top["average_score"], "Performance:", top["performance"])

# Bottom performer
bottom = df.loc[df["average_score"].idxmin()].to_dict()
print("Bottom student:", bottom["name"], "Avg:", bottom["average_score"], "Performance:", bottom["performance"])

# Percentiles
p25 = np.percentile(df["average_score"], 25)
p50 = np.percentile(df["average_score"], 50)
p75 = np.percentile(df["average_score"], 75)
print(f"Percentiles - 25th: {p25:.2f}, 50th(median): {p50:.2f}, 75th: {p75:.2f}")


Top student: Sanya Avg: 96.33 Performance: Excellent
Bottom student: Rahul Avg: 61.0 Performance: Needs Improvement
Percentiles - 25th: 72.50, 50th(median): 81.67, 75th: 89.83


In [0]:
# Correlations
corr_study_avg = df["study_hours"].corr(df["average_score"])
corr_study_math = df["study_hours"].corr(df["math_score"])
print(f"Correlation (study_hours vs average_score): {corr_study_avg:.2f}")
print(f"Correlation (study_hours vs math_score): {corr_study_math:.2f}")

# Correlation matrix (full)
display(df[["math_score","reading_score","writing_score","average_score","study_hours"]].corr().round(2))


Correlation (study_hours vs average_score): 0.96
Correlation (study_hours vs math_score): 0.96


math_score,reading_score,writing_score,average_score,study_hours
1.0,0.93,0.95,0.98,0.96
0.93,1.0,0.98,0.99,0.93
0.95,0.98,1.0,0.99,0.93
0.98,0.99,0.99,1.0,0.96
0.96,0.93,0.93,0.96,1.0


In [0]:
# Convert pandas df back to Spark DataFrame
processed_spark_df = spark.createDataFrame(df)

# Save as a managed table in the same catalog/schema (overwrite if exists)
processed_spark_df.write.mode("overwrite").saveAsTable("retail.default.student_analysis")

print("Saved table: retail.default.student_analysis")


Saved table: retail.default.student_analysis


In [0]:
%sql
-- In a SQL cell or SQL Editor
SELECT * FROM retail.default.student_analysis;


name,gender,math_score,reading_score,writing_score,study_hours,average_score,performance
Riya,female,88,92,85,5,88.33,Excellent
Arjun,male,76,70,72,3,72.67,Good
Priya,female,90,95,93,6,92.67,Excellent
Rahul,male,65,60,58,2,61.0,Needs Improvement
Ananya,female,82,85,88,4,85.0,Excellent
Vikram,male,58,64,62,2,61.33,Needs Improvement
Meera,female,94,91,89,6,91.33,Excellent
Karan,male,70,75,72,3,72.33,Good
Sneha,female,80,83,81,4,81.33,Good
Dev,male,85,78,82,5,81.67,Good


In [0]:
# Convert Pandas DataFrame to Spark
spark_df = spark.createDataFrame(df)

# Save as a managed table
spark_df.write.mode("overwrite").saveAsTable("retail.default.student_analysis_results")

print("✅ Analysis results saved as table: retail.default.student_analysis_results")


✅ Analysis results saved as table: retail.default.student_analysis_results


In [0]:
display(df)

name,gender,math_score,reading_score,writing_score,study_hours,average_score,performance
Riya,female,88,92,85,5,88.33,Excellent
Arjun,male,76,70,72,3,72.67,Good
Priya,female,90,95,93,6,92.67,Excellent
Rahul,male,65,60,58,2,61.0,Needs Improvement
Ananya,female,82,85,88,4,85.0,Excellent
Vikram,male,58,64,62,2,61.33,Needs Improvement
Meera,female,94,91,89,6,91.33,Excellent
Karan,male,70,75,72,3,72.33,Good
Sneha,female,80,83,81,4,81.33,Good
Dev,male,85,78,82,5,81.67,Good


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.